### [기후] 단계적 변수 추출 - LASSO & Elasticnet

In [0]:
# load libraris 
# load libraries 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 
import platform #운영 체제 
import seaborn as sns
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False 

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    path="c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system')

    
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
 

In [0]:
# load data
climate = pd.read_csv('../oniondata/adjusted_climate.csv', encoding='utf-8')
air = pd.read_csv('../oniondata/air_merged.csv', encoding = 'utf-8')
soil = pd.read_csv('../oniondata/soil_final_for_merge.csv', encoding = 'utf-8')
output=pd.read_csv('../oniondata/onion_unit_output.csv', encoding = 'utf-8')

In [0]:
# merge climate and output data into one df 
climate = pd.merge(climate,output,on=['year_local','area','year'])
climate = climate.iloc[:,3:]
climate

,평균기온 7월,평균기온 8월,평균기온 9월,평균기온 10월,평균기온 11월,평균기온 12월,평균기온 1월,평균기온 2월,평균기온 3월,평균기온 4월,...,순간최대풍속 10월,순간최대풍속 11월,순간최대풍속 12월,순간최대풍속 1월,순간최대풍속 2월,순간최대풍속 3월,순간최대풍속 4월,순간최대풍속 5월,순간최대풍속 6월,10a당 생산량 (kg)
0,22.827957,24.297133,18.242593,11.864875,7.723704,-1.206093,-4.854480,-2.994444,2.805018,10.331852,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4061
1,25.051613,25.911828,20.493023,13.147312,8.160000,-0.915054,-4.255914,-1.811905,4.422581,11.940000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4061
2,26.101075,27.240502,21.902222,15.447670,11.091111,3.099642,1.022581,1.350794,6.797849,12.648519,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5121
3,24.853666,26.011144,20.131818,13.698534,9.552424,1.166276,-1.537243,-0.499675,4.974194,11.554545,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6300
4,26.590323,27.625806,22.496667,15.925806,11.573333,3.732258,1.438710,1.353571,7.116129,13.536667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,26.406048,26.978455,21.398333,14.785887,9.688333,4.221023,1.875893,3.102232,7.900446,11.381696,...,9.811694,7.856667,10.423864,10.804018,10.716518,12.120982,9.909375,9.979435,8.046250,6873
431,26.887500,27.388710,20.442917,12.579839,6.805000,1.428977,-0.809375,1.659375,6.600000,11.473661,...,7.349597,5.769583,6.481818,6.341964,6.934821,8.884375,7.380357,7.684274,6.426582,7957
432,25.447312,28.087097,23.584444,17.760215,14.137778,9.384848,6.883333,7.928571,10.902381,14.417857,...,10.611828,8.973333,11.909091,11.352381,10.565476,11.692857,9.042857,8.007527,7.308889,7875
433,24.305161,27.466452,20.138667,12.011613,7.014000,0.330909,-1.005714,0.845714,6.144286,11.113571,...,7.002581,5.654667,6.339091,6.110714,6.544286,8.260000,7.385000,7.576129,6.124000,5500


In [0]:
# confirm if dtype is float for future modeling
climate.dtypes

평균기온 7월          float64
평균기온 8월          float64
평균기온 9월          float64
평균기온 10월         float64
평균기온 11월         float64
                  ...   
순간최대풍속 3월        float64
순간최대풍속 4월        float64
순간최대풍속 5월        float64
순간최대풍속 6월        float64
10a당 생산량 (kg)     object
Length: 121, dtype: object

In [0]:
# transform 10a당 생산량 (kg)s '-' to 0
climate = climate.replace('-', 0)

In [0]:
# Scaling 
climate_columns = climate.columns.tolist()

scaler = MinMaxScaler()
climate_scaled = scaler.fit_transform(climate)
climate_scaled = pd.DataFrame(climate_scaled, columns = climate_columns)

climate_scaled

,평균기온 7월,평균기온 8월,평균기온 9월,평균기온 10월,평균기온 11월,평균기온 12월,평균기온 1월,평균기온 2월,평균기온 3월,평균기온 4월,...,순간최대풍속 10월,순간최대풍속 11월,순간최대풍속 12월,순간최대풍속 1월,순간최대풍속 2월,순간최대풍속 3월,순간최대풍속 4월,순간최대풍속 5월,순간최대풍속 6월,10a당 생산량 (kg)
0,0.390530,0.536933,0.297459,0.151368,0.437209,0.288114,0.219891,0.049930,0.065685,0.316907,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.433497
1,0.574046,0.693454,0.539888,0.275519,0.470633,0.306915,0.257361,0.140811,0.238171,0.501900,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.433497
2,0.660656,0.822250,0.691695,0.498214,0.695182,0.566260,0.587797,0.383871,0.491455,0.583404,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.546648
3,0.557709,0.703081,0.500977,0.328882,0.577305,0.441367,0.427551,0.241659,0.296992,0.457559,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.672502
4,0.701033,0.859600,0.755732,0.544501,0.732125,0.607127,0.613847,0.384085,0.525394,0.685572,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,0.685825,0.796848,0.637413,0.434147,0.587717,0.638700,0.641215,0.518473,0.609029,0.437676,...,0.804822,0.562261,0.661189,0.673757,0.740164,0.817827,0.742832,0.842490,0.750816,0.733668
431,0.725559,0.836617,0.534490,0.220583,0.366828,0.458337,0.473116,0.407586,0.470358,0.448255,...,0.602864,0.412900,0.411144,0.395496,0.478971,0.599446,0.553250,0.648727,0.599681,0.849381
432,0.606702,0.904315,0.872914,0.722088,0.928584,0.972279,0.954682,0.889388,0.929136,0.786939,...,0.870455,0.642176,0.755398,0.707954,0.729732,0.788940,0.677875,0.676017,0.682011,0.840628
433,0.512442,0.844153,0.501715,0.165574,0.382840,0.387403,0.460825,0.345055,0.421763,0.406832,...,0.574400,0.404676,0.402091,0.381075,0.451998,0.557319,0.553598,0.639597,0.571446,0.587105


In [0]:
X = climate_scaled.iloc[:,:-1]
y = climate_scaled.iloc[:,-1]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

##### 1-1. Traning:Validation (70:30) 한 번 적용한 LASSO 모델

In [0]:
#MSE 계산 함수 
def calc_train_error(X_train, y_train, model):
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    return rmse

def calc_validation_error(X_test, y_test, model):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    return rmse

def calc_metrics(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    train_error = calc_train_error(X_train, y_train, model)
    validation_error = calc_validation_error(X_test, y_test, model)
    return train_error, validation_error

#변수 추출 함수
def valid_columns (X_train, y_train, X_test, y_test, model):
    variable = X_train.columns
    coef = pd.DataFrame(pd.Series(model.coef_, variable).sort_values())
    valid_coef = coef[coef.values != 0]
    return valid_coef.index

In [0]:
alphas = [0.0001, 0.0005, 0.001, 0.005, 0.01,0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
#alphas = np.logspace(-4, -0.01, 30)

rmse = []
for alpha in alphas:
    model = Lasso(alpha, random_state = 0)
    rmse.append(calc_metrics(X_train, y_train, X_test, y_test, model)[1])

rmse_table = pd.DataFrame({'alpha':alphas, 'rmse':rmse})
rmse_table = rmse_table.sort_values('rmse')
rmse_table.head()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.02702706409285227, tolerance: 0.001091742470811304
  positive)


,alpha,rmse
2,0.0010,0.176711
3,0.0050,0.176879
1,0.0005,0.177017
0,0.0001,0.184494
4,0.0100,0.186406


In [0]:
model = Lasso(alpha=rmse_table.iloc[0,0], random_state = 0)

print(' best alpha:', rmse_table.iloc[0,0])
print('\n 테스트 RMSE 변수:', calc_metrics(X_train, y_train, X_test, y_test, model))
print('\n 추출 변수:', valid_columns(X_train, y_train, X_test, y_test, model))
print('\n 추출 변수 길이:', len(valid_columns(X_train, y_train, X_test, y_test, model)))


rmse_lasso = calc_metrics(X_train, y_train, X_test, y_test, model)[1]
climate_valid_columns_lasso = valid_columns(X_train, y_train, X_test, y_test, model).values.tolist()

 best alpha: 0.001

 테스트 RMSE 변수: (0.15359485198331718, 0.17671116306556045)

 추출 변수: Index(['습도 11월', '최저기온 5월', '일사량 9월', '평균기온 6월', '일사량 10월', '일조시간 6월',
       '일조시간 10월', '일조시간 2월', '습도 12월', '습도 5월', '평균기온 9월', '최저기온 4월',
       '강수량 7월', '순간최대풍속 8월', '일조시간 5월', '순간최대풍속 7월', '운량 10월', '최고기온 11월',
       '운량 9월', '순간최대풍속 5월', '강수량 10월', '순간최대풍속 12월', '순간최대풍속 3월', '일조시간 8월',
       '평균기온 12월', '최고기온 3월'],
      dtype='object')

 추출 변수 길이: 26


#### 1-2. GridSearch, Cross Validation 적용한 Lasso 모델

In [0]:
param_grid = {'alpha': [0.0001, 0.0005, 0.001, 0.005, 0.01,0.05, 0.1, 0.5, 1, 5, 10, 50, 100]}
#param_grid = {'alpha': np.logspace(-4, -0.01, 30).tolist()}

grid_search_lasso = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search_lasso.fit(X_train, y_train)

print("테스트 RMSE 점수:", np.sqrt(-(grid_search_lasso.score(X_test, y_test))))
print("최적 매개변수:", grid_search_lasso.best_params_)
print("최고 교차 검증 점수:", np.sqrt(-(lasso_regressor.best_score_)))
print("최고 성능 모델:", grid_search_lasso.best_estimator_)

rmse_lasso_cv =  np.sqrt(-(grid_search_lasso.score(X_test, y_test)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1647016978960143, tolerance: 0.0009198561723438001
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1122632380861126, tolerance: 0.0008528001948513563
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.006419738734949831, tolerance: 0.0008768673078074574
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iteratio

테스트 RMSE 점수: 0.17671116306556045
최적 매개변수: {'alpha': 0.001}
최고 교차 검증 점수:{} 0.1767914100666327
최고 성능 모델: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)


In [0]:
variable = X.columns.tolist()
coef = pd.DataFrame(pd.Series(grid_search_lasso.best_estimator_.coef_, variable).sort_values())
valid_coef = coef[coef.values != 0]
climate_valid_columns_lasso_cv = valid_coef.index.values 
valid_coef

,0
습도 11월,-0.056102
최저기온 5월,-0.049046
일사량 9월,-0.046306
평균기온 6월,-0.036964
일사량 10월,-0.027635
일조시간 6월,-0.027280
일조시간 10월,-0.025895
일조시간 2월,-0.025423
습도 12월,-0.024979
습도 5월,-0.014774


##### 2-1. Traning:Validation (70:30) 한 번 적용한 ELASTICNET 모델

In [0]:
alphas = [0.0001, 0.0005, 0.001, 0.005, 0.01,0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
#alphas = np.logspace(-4, -0.01, 30)

rmse = []
for alpha in alphas:
    model = ElasticNet(alpha,l1_ratio=0.5, random_state = 0)
    rmse.append(calc_metrics(X_train, y_train, X_test, y_test, model)[1])

rmse_table = pd.DataFrame({'alpha':alphas, 'rmse':rmse})
rmse_table = rmse_table.sort_values('rmse')
rmse_table.head()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6781942810931332, tolerance: 0.001091742470811304
  positive)


,alpha,rmse
3,0.0050,0.175671
2,0.0010,0.176915
4,0.0100,0.177348
1,0.0005,0.178233
0,0.0001,0.186724


In [0]:
model = ElasticNet(alpha=rmse_table.iloc[0,0], l1_ratio=0.5, random_state = 0)

print(' best alpha:', rmse_table.iloc[0,0])
print('\n Training & Test RMSE 값:', calc_metrics(X_train, y_train, X_test, y_test, model))
print('\n 추출 변수:', valid_columns(X_train, y_train, X_test, y_test, model))
print('\n 추출 변수 길이:', len(valid_columns(X_train, y_train, X_test, y_test, model)))

rmse_elastic = calc_metrics(X_train, y_train, X_test, y_test, model)[1]
climate_valid_columns_elasticnet = valid_columns(X_train, y_train, X_test, y_test, model).values.tolist()

 best alpha: 0.005

 Training & Test RMSE 값: (0.15899414247144025, 0.17567109192770972)

 추출 변수: Index(['일사량 9월', '일사량 10월', '습도 12월', '강수량 7월', '최고기온 6월', '평균기온 2월',
       '최고기온 12월', '운량 9월', '순간최대풍속 2월', '순간최대풍속 3월', '평균기온 3월', '순간최대풍속 12월',
       '일조시간 8월', '순간최대풍속 5월', '최고기온 3월', '평균기온 12월'],
      dtype='object')

 추출 변수 길이: 16


#### 2-1. Cross Validation 적용한 ELASTICNET 모델

In [0]:
param_grid = {'alpha': [0.0001, 0.0005, 0.001, 0.005, 0.01,0.05, 0.1, 0.5, 1, 5, 10, 50, 100]}

grid_search_elastic= GridSearchCV(ElasticNet(), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_elastic.fit(X_train,y_train)

print("테스트 RMSE 점수:", np.sqrt(-(grid_search_elastic.score(X_test, y_test))))
print("최적 매개변수:", grid_search_elastic.best_params_)
print("최고 교차 검증 점수:", np.sqrt(-(grid_search_elastic.best_score_)))
print("최고 성능 모델:", grid_search_elastic.best_estimator_)

rmse_elastic_cv =  np.sqrt(-(grid_search_elastic.score(X_test, y_test)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.61250233650053, tolerance: 0.0009198561723438001
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4954207058562221, tolerance: 0.0008528001948513563
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4001425209480276, tolerance: 0.0008768673078074574
  positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. 

테스트 RMSE 점수: 0.17567109192770972
최적 매개변수: {'alpha': 0.005}
최고 교차 검증 점수: 0.1660472895816242
최고 성능 모델: ElasticNet(alpha=0.005, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)


In [0]:
variable = X.columns.tolist()
coef = pd.DataFrame(pd.Series(grid_search_elastic.best_estimator_.coef_, variable).sort_values())
valid_coef = coef[coef.values != 0]
climate_valid_columns_elasticnet_cv = valid_coef.index.values 
valid_coef

,0
일사량 9월,-0.036215
일사량 10월,-0.024910
습도 12월,-0.016758
강수량 7월,-0.001177
최고기온 6월,-0.000416
평균기온 2월,0.001848
최고기온 12월,0.004443
운량 9월,0.014750
순간최대풍속 2월,0.022982
순간최대풍속 3월,0.049104


### 최적의 변수 추출
- GridSearch가 return하는 prameter값이 같다면, 어차피 모델의 rmse와 모델이 선별해주는 변수의 종류는 같음 

In [0]:
print(rmse_lasso)
print(rmse_lasso_cv)

print(rmse_elastic)
print(rmse_elastic_cv)

0.17671116306556045
0.17671116306556045
0.17567109192770972
0.17567109192770972


In [0]:
print('Lasso을 통해 선별된 기후 변수:', climate_valid_columns_lasso)
print('\nLasso CV 을 통해 선별된 기후 변수:', climate_valid_columns_lasso_cv)

print('\nElasticnet을 통해 선별된 기후 변수:', climate_valid_columns_elasticnet)
print('\nElasticnet CV을 통해 선별된 기후 변수:', climate_valid_columns_elasticnet_cv)

Lasso을 통해 선별된 기후 변수: ['습도 11월', '최저기온 5월', '일사량 9월', '평균기온 6월', '일사량 10월', '일조시간 6월', '일조시간 10월', '일조시간 2월', '습도 12월', '습도 5월', '평균기온 9월', '최저기온 4월', '강수량 7월', '순간최대풍속 8월', '일조시간 5월', '순간최대풍속 7월', '운량 10월', '최고기온 11월', '운량 9월', '순간최대풍속 5월', '강수량 10월', '순간최대풍속 12월', '순간최대풍속 3월', '일조시간 8월', '평균기온 12월', '최고기온 3월']

Lasso CV 을 통해 선별된 기후 변수: ['습도 11월' '최저기온 5월' '일사량 9월' '평균기온 6월' '일사량 10월' '일조시간 6월' '일조시간 10월'
 '일조시간 2월' '습도 12월' '습도 5월' '평균기온 9월' '최저기온 4월' '강수량 7월' '순간최대풍속 8월'
 '일조시간 5월' '순간최대풍속 7월' '운량 10월' '최고기온 11월' '운량 9월' '순간최대풍속 5월' '강수량 10월'
 '순간최대풍속 12월' '순간최대풍속 3월' '일조시간 8월' '평균기온 12월' '최고기온 3월']

Elasticnet을 통해 선별된 기후 변수: ['일사량 9월', '일사량 10월', '습도 12월', '강수량 7월', '최고기온 6월', '평균기온 2월', '최고기온 12월', '운량 9월', '순간최대풍속 2월', '순간최대풍속 3월', '평균기온 3월', '순간최대풍속 12월', '일조시간 8월', '순간최대풍속 5월', '최고기온 3월', '평균기온 12월']

Elasticnet CV을 통해 선별된 기후 변수: ['일사량 9월' '일사량 10월' '습도 12월' '강수량 7월' '최고기온 6월' '평균기온 2월' '최고기온 12월'
 '운량 9월' '순간최대풍속 2월' '순간최대풍속 3월' '평균기온 3월' '순간최대풍속 12월' '일조시간 8월'
 '순간최대풍속 5월' '최고

In [0]:
climate_valid_columns_lasso == climate_valid_columns_lasso_cv

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [0]:
climate_valid_columns_elasticnet == climate_valid_columns_elasticnet_cv

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])